## Figure 3
Distribution of parameter values from MCMC calibration.
Each row shows a different model (OV, SL, RU). Each column shows a different parameter (sigma_org, delta_min / A0, C0, m_sigma).
In the title of each subplot is the best fit and 95% CI for each parameter value.